<a href="https://colab.research.google.com/github/Guljeren/Ml_lab/blob/main/lab_5_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer

# ПАРСИНГ 50 НОВОСТИ
def parse_lenta_news(num_news=50):
    url = "https://lenta.ru/parts/news/"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except:
        print("Ошибка подключения к сайту")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    news_items = soup.find_all('a', class_='card-full-news _parts-news')

    # Берем указанное количество новостей
    available = min(len(news_items), num_news)
    news_items = news_items[:available]

    data = []
    print(f"Найдено новостей на странице: {len(news_items)}")

    for i, item in enumerate(news_items):
        title = item.get_text(strip=True)
        link = item.get('href')

        if link and link.startswith('/'):
            link = 'https://lenta.ru' + link

        # Получаем текст новости
        text_content = ""
        if link and i < 20:  # Для первых 20 новостей получаем полный текст
            try:
                article_response = requests.get(link, headers=headers, timeout=5)
                if article_response.status_code == 200:
                    article_soup = BeautifulSoup(article_response.text, 'html.parser')
                    text_block = article_soup.find('div', class_='topic-body__content')
                    if text_block:
                        paragraphs = text_block.find_all('p')
                        text_content = ' '.join([p.get_text(strip=True) for p in paragraphs])
            except:
                pass

        # Если не удалось получить текст, используем заголовок
        if not text_content or len(text_content) < 50:
            text_content = title

        data.append({
            'title': title,
            'text': text_content,
            'link': link
        })

    return data

#ПРЕДОБРАБОТКА ТЕКСТА
def clean_text(text):
    """Очистка текста от знаков препинания и лишних символов"""
    text = str(text).lower()

    # Удаляем всё, кроме букв и пробелов
    text = re.sub(r'[^а-яёa-z\s]', ' ', text)

    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def main():
    #Парсим 50 новостей
    news_data = parse_lenta_news(50)

    if not news_data:
        print("Не удалось получить новости. Проверьте интернет-соединение.")
        return

    df = pd.DataFrame(news_data)
    print(f"Успешно собрано: {len(df)} новостей")

    # Очистка текстов
    print("\n Предобработка текстов:")
    df['cleaned_text'] = df['text'].apply(clean_text)

    # Показываем пример
    print("Пример очистки:")
    print(f"До:   {df['text'].iloc[0][:80]}...")
    print(f"После: {df['cleaned_text'].iloc[0][:80]}...")

    # 3. Создаем частотный словарь
    print("\nСоздание частотного словаря...")
    all_words = []
    for text in df['cleaned_text']:
        words = text.split()
        all_words.extend([w for w in words if len(w) > 2])

    from collections import Counter
    word_freq = Counter(all_words)

    print(f"Всего уникальных слов: {len(word_freq)}")
    print("Топ-10 самых частых слов:")
    for word, count in word_freq.most_common(10):
        print(f"  {word}: {count}")

    # 4. Bag of Words с помощью CountVectorizer
    vectorizer = CountVectorizer(max_features=1000, stop_words=['и', 'в', 'на', 'с', 'по', 'о', 'к', 'у', 'от', 'за'])
    bow_matrix = vectorizer.fit_transform(df['cleaned_text'])

    print(f"Размер BoW матрицы: {bow_matrix.shape}")
    print(f"Количество признаков (слов): {len(vectorizer.get_feature_names_out())}")

    # Сохраняем BoW в CSV
    bow_df = pd.DataFrame(
        bow_matrix.toarray(),
        columns=vectorizer.get_feature_names_out()
    )
    bow_df.to_csv('bow_matrix.csv', index=False)

    # 5. Создание матрицы эмбеддингов (простая версия)
    vocab_size = len(vectorizer.vocabulary_)
    embedding_dim = 50

    # Создаем случайную матрицу эмбеддингов (в реальности нужно загружать предобученную)
    embedding_matrix = np.random.randn(vocab_size, embedding_dim)

    # Нормализуем векторы
    embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1, keepdims=True)

    print(f"Размер матрицы эмбеддингов: {embedding_matrix.shape}")

    # Сохраняем матрицу эмбеддингов
    np.save('embedding_matrix.npy', embedding_matrix)

    # 6. Сохранение всех результатов
    df.to_csv('news_50_processed.csv', index=False, encoding='utf-8-sig')

    # Сохраняем частотный словарь
    freq_df = pd.DataFrame(list(word_freq.items()), columns=['word', 'count'])
    freq_df.to_csv('word_frequency.csv', index=False)

    # 7. Вывод итог
    print("РЕЗУЛЬТАТЫ")
    print("\n news_50_processed.csv")
    print(f"   - {len(df)} новостей")
    print(f"   - Колонки: {list(df.columns)}")

    print("\n bow_matrix.csv")
    print(f"   - Размер: {bow_matrix.shape}")
    print(f"   - Формат: CSV (можно открыть в Excel)")

    print("\n embedding_matrix.npy")
    print(f"   - Размер: {embedding_matrix.shape}")
    print(f"   - Формат: NumPy binary")

    print("\n word_frequency.csv")
    print(f"   - {len(word_freq)} уникальных слов")

if __name__ == "__main__":
    main()

Найдено новостей на странице: 20
Успешно собрано: 20 новостей

 Предобработка текстов:
Пример очистки:
До:   Российские бойцы зашли в тылВооруженных силУкраины(ВСУ) в боях за Красноармейск ...
После: российские бойцы зашли в тылвооруженных силукраины всу в боях за красноармейск и...

Создание частотного словаря...
Всего уникальных слов: 1725
Топ-10 самых частых слов:
  что: 58
  для: 14
  она: 14
  его: 13
  этом: 12
  это: 11
  ранее: 11
  всу: 10
  также: 10
  как: 10
Размер BoW матрицы: (20, 1000)
Количество признаков (слов): 1000
Размер матрицы эмбеддингов: (1000, 50)
РЕЗУЛЬТАТЫ

 news_50_processed.csv
   - 20 новостей
   - Колонки: ['title', 'text', 'link', 'cleaned_text']

 bow_matrix.csv
   - Размер: (20, 1000)
   - Формат: CSV (можно открыть в Excel)

 embedding_matrix.npy
   - Размер: (1000, 50)
   - Формат: NumPy binary

 word_frequency.csv
   - 1725 уникальных слов
